In [ ]:
from __future__ import absolute_import, division, print_function
import numpy as np
import sympy
import matplotlib.pyplot as plt
from chempy.chemistry import Reaction, Substance, ReactionSystem
from chempy.kinetics.ode import get_odesys
from pyneqsys.plotting import mpl_outside_legend
sympy.init_printing()
%matplotlib inline

In [ ]:
# Never mind the next row, it contains all the reaction data of aqueous radiolysis at room temperature
_species, _reactions = ['H', 'H+', 'H2', 'e-aq', 'HO2-', 'HO2', 'H2O2', 'O-', 'O2', 'O2-', 'O3-', 'OH', 'OH-', 'H2O'], [({3: 2}, {2: 1, 12: 2}, 7259834472.967435, {13: 2}), ({0: 2}, {2: 1}, 5141687640.098337, {}), ({11: 2}, {6: 1}, 4814049818.92958, {}), ({0: 1, 3: 1}, {2: 1, 12: 1}, 27620584849.903786, {13: 1}), ({11: 1, 3: 1}, {12: 1}, 35532223631.79411, {}), ({0: 1, 11: 1}, {13: 1}, 10937407573.68078, {}), ({3: 1, 6: 1}, {11: 1, 12: 1}, 13615735593.986479, {}), ({8: 1, 3: 1}, {9: 1}, 22905112692.006832, {}), ({9: 1, 3: 1}, {12: 2, 6: 1}, 12982165509.204676, {}), ({3: 1, 5: 1}, {4: 1}, 12982165509.204676, {}), ({0: 1, 6: 1}, {11: 1, 13: 1}, 36503386.82158795, {}), ({0: 1, 8: 1}, {5: 1}, 13076581190.923431, {}), ({0: 1, 5: 1}, {11: 2}, 11384254168.518192, {}), ({0: 1, 9: 1}, {4: 1}, 11384254168.518192, {}), ({11: 1, 6: 1}, {13: 1, 5: 1}, 29195422.15773015, {}), ({9: 1, 11: 1}, {8: 1, 12: 1}, 10973581753.227375, {}), ({11: 1, 5: 1}, {8: 1, 13: 1}, 8839961043.22633, {}), ({5: 2}, {8: 1, 6: 1}, 839927.9991826, {}), ({9: 1, 5: 1}, {8: 1, 12: 1, 6: 1}, 100175226.0291507, {13: 1}), ({6: 1}, {8: 1, 13: 2}, 1.297536055993305e-07, {6: 1}), ({11: 1, 4: 1}, {9: 1, 13: 1}, 4067124022.2270365, {}), ({6: 1, 7: 1}, {12: 1, 5: 1}, 4067124022.2270365, {}), ({4: 1, 7: 1}, {9: 1, 12: 1}, 786365799.8899204, {}), ({2: 1, 7: 1}, {0: 1, 12: 1}, 127932793.55517752, {}), ({13: 1}, {1: 1, 12: 1}, 2.469572918533074e-05, {}), ({1: 1, 12: 1}, {13: 1}, 137435350882.69037, {}), ({6: 1}, {1: 1, 4: 1}, 0.0942799172846811, {}), ({1: 1, 4: 1}, {6: 1}, 50221841181.85533, {}), ({12: 1, 6: 1}, {4: 1, 13: 1}, 13303373557.265, {}), ({4: 1, 13: 1}, {12: 1, 6: 1}, 1273381.7524374295, {}), ({11: 1}, {1: 1, 7: 1}, 0.0942799172846811, {}), ({1: 1, 7: 1}, {11: 1}, 50221841181.85533, {}), ({11: 1, 12: 1}, {13: 1, 7: 1}, 13303373557.265, {}), ({13: 1, 7: 1}, {11: 1, 12: 1}, 1273381.7524374295, {}), ({5: 1}, {1: 1, 9: 1}, 772661.4132168075, {}), ({1: 1, 9: 1}, {5: 1}, 50221841181.85533, {}), ({12: 1, 5: 1}, {9: 1, 13: 1}, 13303373557.265, {}), ({9: 1, 13: 1}, {12: 1, 5: 1}, 0.15537766509110776, {}), ({0: 1}, {1: 1, 3: 1}, 5.832241910489462, {}), ({1: 1, 3: 1}, {0: 1}, 20948904196.80398, {}), ({0: 1, 12: 1}, {3: 1, 13: 1}, 24403836.653292242, {}), ({3: 1, 13: 1}, {0: 1, 12: 1}, 15.750969446799516, {}), ({0: 1, 13: 1}, {2: 1, 11: 1}, 4.5752318557708666e-05, {}), ({2: 1, 11: 1}, {0: 1, 13: 1}, 39535182.92510026, {}), ({8: 1, 7: 1}, {10: 1}, 3747413146.2747374, {}), ({10: 1}, {8: 1, 7: 1}, 2616.9169985037124, {})]

In [ ]:
species = [Substance(s) for s in _species]
reactions = [
    Reaction({_species[k]: v for k, v in reac.items()},
             {_species[k]: v for k, v in prod.items()}, param,
             {_species[k]: v for k, v in inact_reac.items()})
    for reac, prod, param, inact_reac in _reactions
]

In [ ]:
# radiolytic yields for gamma radiolysis of neat water
C_H2O = 55.5
YIELD_CONV = 0 #1.0364e-07 # mol * eV / (J * molecules)
prod_rxns = [
    Reaction({'H2O': 1}, {'H+': 1,  'OH-': 1},          0.5  * YIELD_CONV / C_H2O),
    Reaction({'H2O': 1}, {'H+': 1, 'e-aq': 1, 'OH': 1}, 2.6  * YIELD_CONV / C_H2O),
    Reaction({'H2O': 1}, {'H':  2, 'H2O2': 1},          0.66 * YIELD_CONV / C_H2O, {'H2O': 1}),
    Reaction({'H2O': 1}, {'H2': 1, 'H2O2': 1},          0.74 * YIELD_CONV / C_H2O, {'H2O': 1}),
    Reaction({'H2O': 1}, {'H2': 1,   'OH': 2},          0.1  * YIELD_CONV / C_H2O, {'H2O': 1}),
    Reaction({'H2O': 1}, {'H2': 3,  'HO2': 2},          0.04 * YIELD_CONV / C_H2O, {'H2O': 3}),
]

In [ ]:
rsys = ReactionSystem(prod_rxns + reactions, species)
for rxn in rsys.rxns:
    print(str(rxn))  # 1 Gy/s

In [ ]:
odesys = get_odesys(rsys)
odesys.exprs[:3]

In [ ]:
j = odesys.get_jac()
j.shape

In [ ]:
def integrate_and_plot(odesys, c0_dict, integrator, ax=None, zero_conc=0, **kwargs):
    c0 = [c0_dict.get(k, zero_conc) for k in _species]
    out = odesys.integrate(np.logspace(-12, 6), c0, params=rsys.params(), integrator=integrator, **kwargs)
    if ax is None:
        ax = plt.subplot(1, 1, 1)
    odesys.plot_result(plot=ax.plot)
    ax.set_xscale('log'); ax.set_yscale('log')
    mpl_outside_legend(ax, prop={'size': 9})

In [ ]:
# ["H", "H+", "H2", "e-aq", "HO2-", "HO2", "H2O2", "HO3", "O-", "O2", "O2-", "O3", "O3-", "OH", "OH-", "H2O"]
c0_dict = {'H+': 1e-7, 'OH-': 1e-7, 'H2O': 55.5}  # , 'H2O2': 1e-20}
plt.figure(figsize=(16,6))
integrate_and_plot(odesys, c0_dict, integrator='gsl', first_step=1e-14, atol=1e-3, rtol=1e-3)
plt.legend()
plt.ylim([1e-16, 60])

In [ ]:
#%debug

In [ ]:
class Partial(object):
    # functools.partial does not propagate getattr, setattr...
    def __init__(self, cb, **kwargs):
        self._Partial_cb = cb
        self._Partial_kwargs = kwargs
    
    def __call__(self, *args, **kwargs):
        new_kwargs = self._Partial_kwargs.copy()
        new_kwargs.update(kwargs)
        return self._Partial_cb(*args, **new_kwargs)
    
    def __getattr__(self, key):
        if key.startswith('_Partial_'):
            return self.__dict__[key]
        return getattr(self._Partial_cb, key)
    
    def __setattr__(self, key, value):
        if key.startswith('_Partial_'):
            self.__dict__[key] = value
        setattr(self._Partial_cb, key, value)

In [ ]:
from pyodesys.symbolic import ScaledSys
ssys = get_odesys(rsys, SymbolicSys=Partial(ScaledSys, dep_scaling=1e6))

In [ ]:
def integrate_and_plot_scaled(rsys, dep_scaling, *args, **kwargs):
    integrate_and_plot(get_odesys(
            rsys, SymbolicSys=Partial(ScaledSys, dep_scaling=dep_scaling)), *args, **kwargs)

fig, axes = plt.subplots(2, 2, figsize=(14, 14))
solvers = 'cvode', 'odeint', 'scipy', 'scipy'
for idx, ax in enumerate(np.ravel(axes)):
    integrate_and_plot_scaled(rsys, 1e12, c0_dict, solvers[idx], ax=ax, atol=1e-4, rtol=1e-4)

In [ ]:
integrate_and_plot(odesys, c0_dict, 'odeint')

In [ ]:
integrate_and_plot(odesys, c0_dict, 'scipy')

In [ ]:
from pyodesys.symbolic import symmetricsys
logexp = (sympy.log, sympy.exp)
LogLogSys = symmetricsys(logexp, logexp, exprs_process_cb=lambda exprs: [
        sympy.powsimp(expr.expand(), force=True) for expr in exprs])
loglogsys = get_odesys(rsys, SymbolicSys=LogLogSys)

In [ ]:
loglogsys.exprs[:3]

integrate_and_plot(loglogsys, c0_dict, 'gsl', zero_conc=1e-50, first_step=1e-3)